In [2]:
import numpy as np
import pandas as pd

import pyspark
from pydataset import data
from pyspark.sql.functions import *
from pyspark.sql.types import *

## 1. Create a spark data frame that contains your favorite programming languages.
> - The name of the column should be `language`
> - View the schema of the dataframe
> - Output the shape of the dataframe
> - Show the first 5 records in the dataframe

In [3]:
# Create a Pandas dataframe

np.random.seed(123)

pandas_df = pd.DataFrame(
    dict(language=np.random.choice(('Python', 'R', 'C++', 'Fortran', 'SQL'), 20))
)
pandas_df

,language
0,C++
1,SQL
2,C++
3,R
4,Fortran
5,C++
6,Fortran
7,R
8,R
9,Python


In [5]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# Create the spark dataframe
df = spark.createDataFrame(pandas_df)
df

DataFrame[language: string]

In [6]:
# Show the first 5 records
df.show(5)

+--------+
|language|
+--------+
|     C++|
|     SQL|
|     C++|
|       R|
| Fortran|
+--------+
only showing top 5 rows



In [7]:
df.describe()

DataFrame[summary: string, language: string]

In [8]:
df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|      20|
|   mean|    null|
| stddev|    null|
|    min|     C++|
|    max|     SQL|
+-------+--------+



In [12]:
# Show the shape of the dataframe
len(df.columns), df.count()

(1, 20)

In [13]:
# Show the schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



## 2. Load the mpg dataset as a spark dataframe.

> a. Create 1 column of output that contains a message like the one below:
    * ```The 1999 audi a4 has a 4 cylinder engine.```
    
    For each vehicle

In [15]:
# Load the mpg dataset as a sparkdataframe
mpg = spark.createDataFrame(data('mpg'))

# Show the first 5 records
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [18]:
# Build the message by concat the year, man, model, and cyl; and then assign an alias
mpg.select(
    concat(
        lit("The "),  # 'The ' as a literal string
        col("year"),  # the column year
        lit(" "),
        col("manufacturer"),
        lit(" "),
        col("model"),
        lit(" has a "),
        col("cyl"),
        lit(" cylinder engine."),
    ).alias("vehicle_cylinder_desc")
).show(truncate=False) # truncate=False makes it so that the complete sentence for each row is shown

+--------------------------------------------------------------+
|vehicle_cylinder_desc                                         |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

> b. Transform the trans column so that it only contains either manual or auto.

In [20]:
mpg.select('trans', regexp_replace('trans', r'.{4}$', '').alias('trans')).show(5)

+----------+------+
|     trans| trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



## 3. Load the tips dataset as a spark dataframe.
>    a. What percentage of observations are smokers?

>    b. Create a column that contains the tip percentage

>    c. Calculate the average tip percentage for each combination of sex and smoker.

In [22]:
# Load tips data
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### a. What percentage of observations are smokers?

In [27]:
tips.groupby('smoker').count().show(5)

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [29]:
tips.count()

244

In [36]:
# group smokers together, then concat the percentage to the integer
tips.groupby('smoker').count().withColumn(
    'percent',
    concat(round((col('count') / tips.count() * 100), 0).cast('int')),
          ).show(5)

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|     62|
|   Yes|   93|     38|
+------+-----+-------+



### b. Create a column that contains the tip percentage

In [40]:
# Calculate the tip percentage
#df = tips.withColumn('tip_percent',
#                concat(round(col('tip') / col('total_bill') * 100, 2)))
#df.show()

# alternative method
# calculate tip percentage
tip_percentage = tips.tip / tips.total_bill
# create column for tip percentage
# must create alias or column title will be tip / total_bill
tips.select('*', tip_percentage.alias('tip_percent')).show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|        tip_percent|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



### c. Calculate the average tip percentage for each combination of sex and smoker.

In [9]:
df.groupby('smoker').pivot('sex').agg({'tip_percent': 'mean'}).show()

+------+------------------+------------------+
|smoker|            Female|              Male|
+------+------------------+------------------+
|    No| 15.69111111111111| 16.06659793814433|
|   Yes|18.214545454545455|15.276666666666667|
+------+------------------+------------------+



##  Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [42]:
from vega_datasets import data
# Import and create the spark dataframe
weather = spark.createDataFrame(data.seattle_weather())
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



### A. Convert the temperatures to farenheight.


In [46]:
# celcius to farenheight -> (C*1.8)+32
weather = weather.withColumn('temp_max', round((col('temp_max') * 1.8 + 32), 2)
                ).withColumn('temp_min', round((col('temp_max') * 1.8 + 32), 2))

weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|  267.93|  514.27| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|  255.09|  491.16| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|  261.52|  502.74| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|  264.43|  507.97| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|  245.19|  473.34| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



### B. Which month has the most rain, on average?

In [20]:

row = (
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
    .first()
)
print(row.month)
print(row.avg_monthly_rain)

11
160.625


### C. Which year was the windiest?

In [50]:
max_wind = weather.withColumn("year", year("date")).groupBy("year").agg(sum("wind").alias("total_winds")).sort(col("total_winds").desc()).collect()[0]
    
max_wind

Row(year=2012, total_winds=1244.6999999999998)

### D. What is the most frequent type of weather in January?

In [32]:
weather_type = (weather.withColumn("month", month("date"))
    .filter(col("month") == 1) # This is the month of jav
    .groupBy("weather") # Group by the type of weather
    .count() # Count the frequency
    .sort(col("count").desc()) # Sort the frequencies
    .collect()[0]) # this selects the first row
    
weather_type

Row(weather='fog', count=38)

### E. What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [35]:
avg_h_l = (weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max").alias("average_high_temp"),
        avg("temp_min").alias("average_low_temp"),
    )
    .show())

+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



### F. What percentage of days were rainy in q3 of 2015?

In [37]:
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



### G. For each year, find what percentage of days it rained (had non-zero precipitation).

In [38]:
(
    weather.withColumn("year", year("date"))
    .select(when(col("precipitation") > 0, 1).otherwise(0).alias("rain"), "year")
    .groupby("year")
    .agg(mean("rain"))
    .show()
)

+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

